I (We) certify that the code and data in this assignment were generated independently, 
using only the tools and resources defined in the course and that I (we) did not receive 
any external help, coaching or contributions during the production of this work.

- Arpan Shaileshbhai Korat
- Om Hemant Purohit

In [1]:
from keras.datasets import fashion_mnist
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
import pandas as pd
tempdf=pd.concat([pd.DataFrame(trainy),pd.DataFrame(testy)])
tempdf["labels"]=tempdf.values.flatten()

# We can see that we have 7k labels for each number from 0 to 9 included

In [ ]:
tempdf.groupby(["labels"]).count()

In [ ]:
tempdf.groupby(["labels"]).count().reset_index()[0]

In [ ]:
import matplotlib.pyplot as plt
gdf=tempdf.groupby(["labels"]).count().reset_index()
plt.bar(gdf["labels"],gdf[0])

In [ ]:
plt.imshow(trainX[0])

0 	T-shirt/top
1 	Trouser
2 	Pullover
3 	Dress
4 	Coat
5 	Sandal
6 	Shirt
7 	Sneaker
8 	Bag
9 	Ankle boot

In [ ]:
def plotClothes(x):
    tempList=[]
    for i in range(len(trainy)):
        if len(tempList)>=4:
            break
        if trainy[i]==x:
            tempList.append(trainX[i])
    fig,ax=plt.subplots(2,2,figsize=(8,8))
    for i in range(4):
        ax[i//2][i%2].imshow(tempList[i])
    




# Plot some t shirts

In [ ]:
plotClothes(0)


# Plot some shoes


In [ ]:
plotClothes(9)

# Plot some sandals

In [ ]:
plotClothes(5)

In [2]:
from keras.datasets import fashion_mnist
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
trainY.shape

In [10]:
trainX=np.expand_dims(trainX,-1)
testX=np.expand_dims(testX,-1)
trainY=keras.utils.to_categorical(trainy,10)
testY=keras.utils.to_categorical(testy,10)


In [ ]:
trainY

In [ ]:
cnn = keras.Sequential([
    keras.Input(shape=(28,28, 1)),
    keras.layers.Conv2D(filters=64, kernel_size=(5, 3), activation='relu',),
    keras.layers.AveragePooling2D((4, 4)),
    
    keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    keras.layers.Conv2D(filters=38, kernel_size=(3, 3), activation='relu'),

    # keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    # keras.layers.Dense(32, activation='relu'),
    # keras.layers.Dense(16, activation='sigmoid'),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
cnn.compile(optimizer=Adam(lr = 0.00010),
  loss='categorical_crossentropy',
              metrics=['accuracy'])

temp=cnn.fit(trainX,trainY,validation_split=0.2,epochs = 90)

In [ ]:
temp.history.keys()

In [ ]:
cnn.summary()

In [ ]:
import matplotlib.pyplot as plt
basetraining=temp.history["accuracy"]
testing=temp.history["val_accuracy"]
fig,ax=plt.subplots(1,1,figsize=(12,9))
ax.plot(testing,label="testing accuracy")
ax.plot(basetraining,label="training accuracy")
ax.legend()
ax.set_title("Training and testing accuracy plot")
ax.set_xlabel("epochs")

In [ ]:
cnn.evaluate(testX,testY)

In [ ]:
basetrainingLoss=temp.history["loss"]
testing=temp.history["val_loss"]
fig,ax=plt.subplots(1,1,figsize=(12,9))
ax.plot(testing,label="testing loss")
ax.plot(basetraining,label="training loss")
ax.legend()
ax.set_title("Training and testing loss plot")
ax.set_xlabel("epochs")

# EarlyStopping

In [ ]:
from gc import callbacks
cnn.compile(optimizer=Adam(lr = 0.00010),
  loss='categorical_crossentropy',
              metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
cnn.fit(trainX,trainY,validation_split=0.2,epochs = 90,callbacks=[callback])

In [ ]:
cnn.evaluate(testX,testY)

# L1-L2 regularization

In [ ]:
from tensorflow.keras import regularizers
cnn = keras.Sequential([
    keras.Input(shape=(28,28, 1)),
    keras.layers.Conv2D(filters=64, kernel_size=(5, 3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.4, l2=0.2)),
    keras.layers.AveragePooling2D((4, 4)),
    
    keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.4, l2=0.2)),
    keras.layers.Conv2D(filters=38, kernel_size=(3, 3), activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.4, l2=0.2)),

    # keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    # keras.layers.Dense(32, activation='relu'),
    # keras.layers.Dense(16, activation='sigmoid'),
    keras.layers.Dense(10, activation='softmax')

])

In [ ]:
cnn.compile(optimizer=Adam(lr = 0.00010),
  loss='categorical_crossentropy',
              metrics=['accuracy'])
cnn.fit(trainX,trainY,validation_split=0.2,epochs = 90)

# K-FOLD Split will not work here as it works for binary values and we are doing multiclass so it will not be applicable here


# Altering Dropout

In [ ]:
dropList=[0.5,0.7,0.9]
testList=[]
trainPlot=[]
testPlot=[]
for d in dropList:
    
    cnn = keras.Sequential([
        keras.Input(shape=(28,28, 1)),
        keras.layers.Conv2D(filters=64, kernel_size=(5, 3), activation='relu',),
        keras.layers.AveragePooling2D((4, 4)),

        keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        keras.layers.Conv2D(filters=38, kernel_size=(3, 3), activation='relu'),

        # keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Flatten(),
        keras.layers.Dropout(d),
        # keras.layers.Dense(32, activation='relu'),
        # keras.layers.Dense(16, activation='sigmoid'),
        keras.layers.Dense(10, activation='softmax')
    ])
    cnn.compile(optimizer=Adam(lr = 0.00010),
      loss='categorical_crossentropy',
                  metrics=['accuracy'])
    temp=cnn.fit(trainX,trainY,validation_split=0.2,epochs = 40)
    testList.append(cnn.evaluate(testX,testY))
    testPlot.append(temp.history["accuracy"])
    trainPlot.append(temp.history["val_accuracy"])

In [ ]:
testList

In [ ]:
import matplotlib.pyplot as plt
optList=["0.5","0.7","0.9"]
fig,ax=plt.subplots(1,3,figsize=(15,8))
for i in range(3):
    ax[i].plot(testPlot[i],label="training accuracy")
    ax[i].plot(trainPlot[i],label="testing accuracy")
   
    ax[i].legend()
    ax[i].set_title("Dropout->" +optList[i])
    ax[i].set_xlabel("epochs")

# Changing Optimizer

In [ ]:
from tensorflow.keras.optimizers import SGD,Nadam
optList=[Adam(lr = 0.00010),Nadam(lr = 0.00010),SGD(lr=0.00010)]
testList=[]
testPlot=[]
trainPlot=[]
for d in optList:
    
    cnn = keras.Sequential([
        keras.Input(shape=(28,28, 1)),
        keras.layers.Conv2D(filters=64, kernel_size=(5, 3), activation='relu',),
        keras.layers.AveragePooling2D((4, 4)),

        keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        keras.layers.Conv2D(filters=38, kernel_size=(3, 3), activation='relu'),

        # keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        # keras.layers.Dense(32, activation='relu'),
        # keras.layers.Dense(16, activation='sigmoid'),
        keras.layers.Dense(10, activation='softmax')
    ])
    cnn.compile(optimizer=d,
      loss='categorical_crossentropy',
                  metrics=['accuracy'])
    temp=cnn.fit(trainX,trainY,validation_split=0.2,epochs = 40)
    testPlot.append(temp.history["accuracy"])
    trainPlot.append(temp.history["val_accuracy"])
    testList.append(cnn.evaluate(testX,testY))

In [ ]:
testList

In [ ]:
import matplotlib.pyplot as plt
optList=["Adam","Nadam","SGD"]
fig,ax=plt.subplots(1,3,figsize=(15,8))
for i in range(3):
    ax[i].plot(testPlot[i],label="training accuracy")
    ax[i].plot(trainPlot[i],label="testing accuracy")
   
    ax[i].legend()
    ax[i].set_title("Optimizer Function->" +optList[i])
    ax[i].set_xlabel("epochs")

# Changing Activation Functions

In [ ]:
from tensorflow.keras.optimizers import SGD,Nadam
optList=['relu','selu','tanh']
testList=[]
testPlot=[]
trainPlot=[]
for d in optList:
    
    cnn = keras.Sequential([
        keras.Input(shape=(28,28, 1)),
        keras.layers.Conv2D(filters=64, kernel_size=(5, 3), activation=d,),
        keras.layers.AveragePooling2D((4, 4)),

        keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation=d),
        keras.layers.Conv2D(filters=38, kernel_size=(3, 3), activation=d),

        # keras.layers.MaxPooling2D((2, 2)),

        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        # keras.layers.Dense(32, activation='relu'),
        # keras.layers.Dense(16, activation='sigmoid'),
        keras.layers.Dense(10, activation='softmax')
    ])
    cnn.compile(optimizer=Adam(lr = 0.00010),
      loss='categorical_crossentropy',
                  metrics=['accuracy'])
    temp=cnn.fit(trainX,trainY,validation_split=0.2,epochs = 40)
    testList.append(cnn.evaluate(testX,testY))
    testPlot.append(temp.history["accuracy"])
    trainPlot.append(temp.history["val_accuracy"])
    

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,3,figsize=(15,8))
for i in range(3):
    ax[i].plot(testPlot[i],label="training accuracy")
    ax[i].plot(trainPlot[i],label="testing accuracy")
   
    ax[i].legend()
    ax[i].set_title("Activation Function->" +optList[i])
    ax[i].set_xlabel("epochs")
    

# Data Augmentation

In [43]:
from keras.datasets import fashion_mnist

(trainX, trainy), (testX, testy) = fashion_mnist.load_data()
newTrainX=trainX.copy()
newTrainY=trainy.copy()


# INCREASING THE DATASET SIZE TO 4 TIMES!
for i in range(3):
    newTrainX=np.concatenate([newTrainX,trainX])
    newTrainY=np.concatenate([newTrainY,trainy])
print(newTrainX.shape)   
    
    
newTrainX=np.expand_dims(newTrainX,-1)

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
  
    horizontal_flip=True,
    )

datagen.fit(newTrainX)




(240000, 28, 28)


In [44]:
newTrainY=keras.utils.to_categorical(newTrainY,10)
testY=keras.utils.to_categorical(testy,10)
countt=0
    
    

In [50]:
cnn = keras.Sequential([
    keras.Input(shape=(28,28, 1)),
    keras.layers.Conv2D(filters=64, kernel_size=(2, 2), activation='relu',),
    keras.layers.AveragePooling2D((2, 2)),
    
    keras.layers.Conv2D(filters=128, kernel_size=(2, 2), activation='relu'),

    # keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    # keras.layers.Dense(32, activation='relu'),
    # keras.layers.Dense(16, activation='sigmoid'),
    keras.layers.Dense(10, activation='softmax')
])

In [54]:
cnn.compile(optimizer=keras.optimizers.Adam(lr = 0.000070),
  loss='categorical_crossentropy',
              metrics=['accuracy'])


temp=cnn.fit(datagen.flow(newTrainX, newTrainY, batch_size=32 ),shuffle=False,epochs = 10)

2022-04-17 14:24:39.691343: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-04-17 14:24:41.420338: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


7500/7500 [==============================] - 99s 12ms/step - loss: 0.6212 - accuracy: 0.7792
Epoch 2/10
7500/7500 [==============================] - 92s 12ms/step - loss: 0.4658 - accuracy: 0.8340
Epoch 3/10
7500/7500 [==============================] - 91s 12ms/step - loss: 0.4175 - accuracy: 0.8518
Epoch 4/10
7500/7500 [==============================] - 93s 12ms/step - loss: 0.3871 - accuracy: 0.8620
Epoch 5/10
7500/7500 [==============================] - 92s 12ms/step - loss: 0.3666 - accuracy: 0.8697
Epoch 6/10
7500/7500 [==============================] - 92s 12ms/step - loss: 0.3514 - accuracy: 0.8741
Epoch 7/10
7500/7500 [==============================] - 92s 12ms/step - loss: 0.3386 - accuracy: 0.8788
Epoch 8/10
7500/7500 [==============================] - 91s 12ms/step - loss: 0.3282 - accuracy: 0.8819
Epoch 9/10
7500/7500 [==============================] - 92s 12ms/step - loss: 0.3201 - accuracy: 0.8844
Epoch 10/10
7500/7500 [==============================] - 91s 12ms/step - lo

In [57]:
newdatagen=tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
  
    horizontal_flip=True,
    )
testXX=np.expand_dims(testX,-1)
newdatagen.fit(testXX)
cnn.evaluate(newdatagen.flow(testXX,testY))

313/313 [==============================] - 4s 11ms/step - loss: 0.3288 - accuracy: 0.8847


[0.3287602961063385, 0.8847000002861023]

In [ ]:
datagen.flow(newTrainX, newTrainY, batch_size=32 )
